In [1]:
!pip install -q crewai langchain langchain_core langchain_community langchain-openai langchain-experimental wikipedia "wikibase-rest-api-client<0.2" mediawikiapi arxiv

In [2]:
from pydantic import BaseModel, Field
from typing import List, Optional, Dict

class CodeInput(BaseModel):
    code_snippet: str
    language: str = Field(default="python", description="Programming language of the code snippet")
    context: Optional[str] = Field(default=None, description="Additional context or comments about the code")
    dependencies: Optional[List[str]] = Field(default=None, description="List of dependencies or libraries used")
    environment: Optional[str] = Field(default=None, description="Execution environment details")
    expected_behavior: Optional[str] = Field(default=None, description="Description of the expected behavior of the code")
    actual_behavior: Optional[str] = Field(default=None, description="Description of the actual behavior observed")
    inputs: Optional[List[str]] = Field(default=None, description="Expected inputs to the code")
    outputs: Optional[List[str]] = Field(default=None, description="Expected outputs from the code")

class BugDetail(BaseModel):
    bug_id: int
    description: str
    line_number: Optional[int]
    severity: str
    error_message: Optional[str]
    stack_trace: Optional[str]
    variables_at_fault: Optional[List[str]]
    conditions: Optional[str]
    frequency: Optional[str]
    module: Optional[str]
    function: Optional[str]
    code_snippet: Optional[str]
    replication_steps: Optional[List[str]]
    logs: Optional[List[str]]
    environment: Optional[str]
    timestamp: Optional[str]

class AnalysisOutput(BaseModel):
    bugs: List[BugDetail]
    warnings: Optional[List[str]]
    error_counts: Optional[Dict[str, int]]
    execution_time: Optional[float]
    resource_usage: Optional[Dict[str, float]]
    code_complexity: Optional[float]
    code_coverage: Optional[float]
    test_results: Optional[Dict[str, str]]
    static_analysis_reports: Optional[Dict[str, str]]
    dependency_issues: Optional[List[str]]

class DebuggingStep(BaseModel):
    step_id: int
    description: str
    related_bug_ids: List[int]
    priority: str
    estimated_time_hours: Optional[float]
    required_resources: Optional[List[str]]
    dependencies: Optional[List[int]]
    assigned_to: Optional[str]
    tools_needed: Optional[List[str]]
    risk_assessment: Optional[str]
    success_criteria: Optional[str]

class DebuggingPlan(BaseModel):
    steps: List[DebuggingStep]
    overall_priority: str
    total_estimated_time_hours: Optional[float]
    risks: Optional[List[str]]
    assumptions: Optional[List[str]]
    contingencies: Optional[List[str]]

class FixSuggestion(BaseModel):
    suggestion_id: int
    bug_id: int
    suggestion: str
    detailed_steps: List[str]
    code_changes: Optional[Dict[str, str]]  # e.g., { "line 42": "change x to y" }
    potential_side_effects: Optional[List[str]]
    testing_required: Optional[List[str]]
    estimated_effort_hours: Optional[float]
    dependencies_affected: Optional[List[str]]
    alternative_solutions: Optional[List[str]]
    references: Optional[List[str]]

class FixSuggestions(BaseModel):
    suggestions: List[FixSuggestion]

class FixedCode(BaseModel):
    code_snippet: str
    changes_made: Dict[str, str]
    bugs_fixed: List[int]
    new_dependencies: Optional[List[str]]
    tests_performed: Optional[List[str]]
    performance_improvements: Optional[Dict[str, float]]
    remaining_issues: Optional[List[str]]
    code_quality_metrics: Optional[Dict[str, float]]
    documentation_updates: Optional[List[str]]

In [3]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [4]:
from crewai import Agent
from textwrap import dedent
from langchain_openai import ChatOpenAI
from langchain_experimental.tools import PythonREPLTool
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_community.tools.wikidata.tool import WikidataAPIWrapper, WikidataQueryRun
from langchain_community.utilities import ArxivAPIWrapper
from langchain.tools import Tool

class CustomAgents:
    def __init__(self):
        self.OpenAIGPT4Mini = ChatOpenAI(model="gpt-4o-mini", temperature=0)
        self.OpenAIGPT4 = ChatOpenAI(model="gpt-4o", temperature=0)

    def bug_finder_agent(self):
        # Agent 1: Bug Finder
        tools = [PythonREPLTool()]
        return Agent(
            role="Bug Finder",
            backstory=dedent("""You are an expert in identifying bugs in code. You can detect syntax errors, runtime errors, logical errors, and any unexpected behavior."""),
            goal=dedent("""Examine the provided code and identify any bugs, errors, or anomalies. Provide detailed information about each bug found."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def bug_analyzer_agent(self):
        # Agent 2: Bug Analyzer
        tools = [
            WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
            WikidataQueryRun(api_wrapper=WikidataAPIWrapper()),
            Tool(
                name="Arxiv",
                func=ArxivAPIWrapper().run,
                description="A wrapper around Arxiv. Useful for accessing academic papers."
            )
        ]
        return Agent(
            role="Bug Analyzer",
            backstory=dedent("""You specialize in analyzing bugs to determine their root causes and potential fixes."""),
            goal=dedent("""Analyze the identified bugs, determine their root causes, and assess their impact on the overall code."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

    def fix_planner_agent(self):
        # Agent 3: Fix Planner
        tools = [
            WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper()),
            WikidataQueryRun(api_wrapper=WikidataAPIWrapper()),
            Tool(
                name="Arxiv",
                func=ArxivAPIWrapper().run,
                description="A wrapper around Arxiv. Useful for accessing academic papers."
            )
        ]
        return Agent(
            role="Fix Planner",
            backstory=dedent("""You provide detailed plans on how to fix the identified bugs, including estimated effort and any affected dependencies."""),
            goal=dedent("""Develop a step-by-step plan to fix the bugs, including priorities, estimated time, and required resources."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4Mini,
        )

    def code_fixer_agent(self):
        # Agent 4: Code Fixer
        tools = [PythonREPLTool()]
        return Agent(
            role="Code Fixer",
            backstory=dedent("""You apply fixes to the code based on the debugging plan and produce the fixed code along with a summary of changes made."""),
            goal=dedent("""Apply the fixes to the code and produce the fixed code along with a summary of the changes made and any new dependencies introduced."""),
            tools=tools,
            allow_delegation=False,
            verbose=True,
            llm=self.OpenAIGPT4,
        )

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


In [5]:
from crewai import Task
from textwrap import dedent

class CustomTasks:
    def __init__(self):
        pass

    def bug_finding_task(self, agent, code_input: CodeInput):
        analysis_output_schema = AnalysisOutput.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Examine the following {code_input.language} code snippet and identify any bugs, errors, or anomalies.
                Provide detailed information about each bug found.
                Provide your output in **JSON format** matching the **AnalysisOutput** schema.

                **Format**:

    json
{analysis_output_schema}

                **Code**:

    {code_input.language}
    {code_input.code_snippet}

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The analysis output in JSON format matching the schema: {analysis_output_schema}",
        )

    def bug_analysis_task(self, agent, code_input: CodeInput):
        debugging_plan_schema = DebuggingPlan.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Based on the identified bugs in the code, analyze each bug to determine its root cause and assess its impact.
                Provide your analysis in **JSON format** matching the **DebuggingPlan** schema.

                **Format**:

    json
{debugging_plan_schema}

                **Code**:

    {code_input.language}
    {code_input.code_snippet}

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The debugging plan in JSON format matching the schema: {debugging_plan_schema}",
        )

    def fix_planning_task(self, agent, code_input: CodeInput):
        fix_suggestions_schema = FixSuggestions.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Develop detailed suggestions for fixing the identified bugs in the following code.
                Include estimated effort in hours for each suggestion.
                Provide your output in **JSON format** matching the **FixSuggestions** schema.

                **Format**:

    json
{fix_suggestions_schema}

                **Code**:

    {code_input.language}
    {code_input.code_snippet}

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The fix suggestions in JSON format matching the schema: {fix_suggestions_schema}",
        )

    def code_fixing_task(self, agent, code_input: CodeInput):
        fixed_code_schema = FixedCode.schema_json(indent=2)
        return Task(
            description=dedent(f"""
                Apply the fix suggestions to the following code.
                Ensure the code remains functional and free of the identified bugs.
                Provide the fixed code and a summary of changes made in **JSON format** matching the **FixedCode** schema.

                **Format**:

    json
{fixed_code_schema}

                **Original Code**:

    {code_input.language}
    {code_input.code_snippet}

                **Additional Context**:
                {code_input.context if code_input.context else 'N/A'}
            """),
            agent=agent,
            expected_output=f"The fixed code in JSON format matching the schema: {fixed_code_schema}",
        )


In [6]:
import os
from crewai import Crew
from textwrap import dedent

# Assuming the schemas and agents are imported from their respective modules
# from schemas import CodeInput
# from agents import CustomAgents
# from tasks import CustomTasks

class DebuggingAssistantCrew:
    def __init__(self, code_snippet, language="python", context=None):
        self.code_input = CodeInput(code_snippet=code_snippet, language=language, context=context)
        self.agents = CustomAgents()
        self.tasks = CustomTasks()

    def run(self):
        # Define agents
        bug_finder_agent = self.agents.bug_finder_agent()
        bug_analyzer_agent = self.agents.bug_analyzer_agent()
        fix_planner_agent = self.agents.fix_planner_agent()
        code_fixer_agent = self.agents.code_fixer_agent()

        # Define tasks
        bug_finding_task = self.tasks.bug_finding_task(bug_finder_agent, self.code_input)
        bug_analysis_task = self.tasks.bug_analysis_task(bug_analyzer_agent, self.code_input)
        fix_planning_task = self.tasks.fix_planning_task(fix_planner_agent, self.code_input)
        code_fixing_task = self.tasks.code_fixing_task(code_fixer_agent, self.code_input)

        # Create the crew
        crew = Crew(
            agents=[
                bug_finder_agent,
                bug_analyzer_agent,
                fix_planner_agent,
                code_fixer_agent,
            ],
            tasks=[
                bug_finding_task,
                bug_analysis_task,
                fix_planning_task,
                code_fixing_task,
            ],
            verbose=True,
        )

        result = crew.kickoff()
        return result

if __name__ == "__main__":
    print("## Welcome to the Multi-Agent Debugging Assistant")
    print("-------------------------------------------------------")
    code_snippet = input(dedent("""Please enter the code you wish to debug:\n"""))
    language = input("Enter the programming language (default is Python): ") or "python"
    context = input("Any additional context or comments about the code? ") or None

    crew = DebuggingAssistantCrew(code_snippet, language, context)
    results = crew.run()
    print("\n\n########################")
    print("## Here is your debugging assistant result:")
    print("########################\n")
    print(results)


## Welcome to the Multi-Agent Debugging Assistant
-------------------------------------------------------
Please enter the code you wish to debug:
    def quicksort(arr):         if len(arr) == 1:             return arr # This will fail for an empty array, causing a crash         pivot = arr[0]         left = []         right = []         for i in arr:             if i < pivot:                 left.append(pivot)             else:                 right.append(pivot)         return quicksort(left) + quicksort([pivot]) + quicksort(right)      arr = [3, 6, 8, 10, 1, 2, 1]     sorted_arr = quicksort(arr)     print(sorted_arr)
Enter the programming language (default is Python): python
Any additional context or comments about the code? This is an incorrect Quicksort algorithm that needs major refactoring and fixes.
# Agent: Bug Finder
## Task: 
                Examine the following python code snippet and identify any bugs, errors, or anomalies.
                Provide detailed information ab



# Agent: Bug Finder
## Thought: I need to analyze the provided Python code snippet for any bugs, errors, or anomalies. I will look for syntax errors, runtime errors, logical errors, and any unexpected behavior in the code.
The code snippet provided is:
```python
def quicksort(arr):         
    if len(arr) == 1:             
        return arr # This will fail for an empty array, causing a crash         
    pivot = arr[0]         
    left = []         
    right = []         
    for i in arr:             
        if i < pivot:                 
            left.append(pivot)             
        else:                 
            right.append(pivot)         
    return quicksort(left) + quicksort([pivot]) + quicksort(right)      
arr = [3, 6, 8, 10, 1, 2, 1]     
sorted_arr = quicksort(arr)     
print(sorted_arr)
```
I will execute the code to observe its behavior and identify any issues.
## Using tool: Python_REPL
## Tool Input: 
"{\"query\": \"def quicksort(arr):\\n    if len(arr

In [8]:
results

CrewOutput(raw='```json\n{\n  "code_snippet": "def quicksort(arr):\\n    if len(arr) == 0 or len(arr) == 1:\\n        return arr\\n    pivot = arr[0]\\n    left = []\\n    right = []\\n    for i in arr[1:]:\\n        if i < pivot:\\n            left.append(i)\\n        else:\\n            right.append(i)\\n    return quicksort(left) + [pivot] + quicksort(right)\\n\\narr = [3, 6, 8, 10, 1, 2, 1]\\nsorted_arr = quicksort(arr)\\nprint(sorted_arr)",\n  "changes_made": {\n    "base_case": "Modified the base case to handle empty arrays by checking if the length of the array is 0 or 1.",\n    "partition_logic": "Corrected the partitioning logic to append the current element \'i\' to the left or right list instead of the pivot."\n  },\n  "bugs_fixed": [1, 2],\n  "new_dependencies": null,\n  "tests_performed": [\n    "Tested with an empty array to ensure it returns an empty array.",\n    "Tested with various arrays, including those with duplicate elements, to ensure correct sorting."\n  ],\n  "